In [55]:
import os
import logging
import requests
from dotenv import load_dotenv
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes

load_dotenv()
def getBearer(apikey):
    form = {
        'apikey': apikey,
        'grant_type': "urn:ibm:params:oauth:grant-type:apikey"
    }
    logging.info("About to create bearer")
    response = requests.post(
        "https://iam.cloud.ibm.com/oidc/token",
        data=form
    )
    if response.status_code != 200:
        logging.error("Bad response code retrieving token")
        raise Exception("Failed to get token, invalid status")
    json = response.json()
    if not json:
        logging.error("Invalid/no JSON retrieving token")
        raise Exception("Failed to get token, invalid response")
    logging.info("Bearer retrieved")
    return json.get("access_token")


load_dotenv()
parameters = {
    GenParams.DECODING_METHOD: "greedy",
    GenParams.MAX_NEW_TOKENS: 1000,
    GenParams.STOP_SEQUENCES: ["\n\n\n"]
}
project_id = os.getenv("PROJECT_ID", None)
credentials = {
    #"url": "https://eu-de.ml.cloud.ibm.com",
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": os.getenv("API_KEY", None)
}
credentials["token"] = getBearer(credentials["apikey"])
model_id = ModelTypes.LLAMA_2_70B_CHAT
# Initialize the Watsonx foundation model
llm_model = Model(
    model_id=model_id,
    params=parameters,
    credentials=credentials,
    project_id=project_id
)


def generate_code(code, language):
    code_prompts = []
    code_prompt = f"""You are really good at debugging in {language} and fixing {language} code. The result is only the code."""
    
    inst_prompt = f"""<s>[INST] <<SYS>> 
        {code_prompt}
        <</SYS>> 
        Print a fixed code in {language} from the following input code: {code}
        [/INST]  Code:"""
    
    code_prompts.append(inst_prompt)
    logging.info("Sending prompt for code")
    result = llm_model.generate(
        code_prompts
    )
    code = ""
    for item in result:
        code += item['results'][0]['generated_text']
    logging.info("Obtained result")
    return code.strip()

def get_chatbot_suggestion(error, code):
    # Generate code fix using CodeGenerator
    generated_code = generate_code(code, "Python")
    # Append the generated code to the suggestion
    suggestion = f"\n\nGenerated Code Fix:\n{generated_code}\n"
    return suggestion

In [56]:
error="""
  File "<stdin>", line 1
    "Hello world
    ^
SyntaxError: unterminated string literal (detected at line 1)

"""
code="""
"Hello world
"""

In [57]:
get_chatbot_suggestion(error, code)

'\n\nGenerated Code Fix:\n```\nprint("Hello world")\n```\n\nFixed code:\n```\nprint("Hello world")\n```\n\nExplanation:\n\nThe code provided is already correct, so there is no need to fix anything. The `print` statement is properly indented and the string "Hello world" is properly enclosed in quotes. Therefore, the code should work as intended and print "Hello world" to the console.\n'

In [58]:
input_string = get_chatbot_suggestion(error, code)


In [59]:
import subprocess
from utils import *
def run_code(code):
    try:
        output = subprocess.check_output(["python", "-c", code], stderr=subprocess.STDOUT)
        return True, output.decode("utf-8")
    except subprocess.CalledProcessError as e:
        return False, e.output.decode("utf-8")

In [60]:
def debug_code(code_input):
    code = code_input
    attempt = 1
    success=False
    while success == False and attempt < 3:
        #output_zone.write(f"Attempt {attempt}: Running code...")
        print(f"Attempt {attempt}: Running code...")
        success, output = run_code(code)
        if success:
            #output_zone.write(f"Code executed successfully. Output: {output}")
            print((f"Code executed successfully. Output: {output}"))
            break
        else:
            error = output
            suggestion = get_chatbot_suggestion(error, code)  # Use your chatbot API to get the suggestion
            code = suggestion  # Update the code with the fixed version from the chatbot
            #fixed_code.write(f"Suggested fix applied:\n{code}")
            print(f"Suggested fix applied:\n{code}")
            #output_zone.write(f"Trying again with the fixed code...")
            print(f"Trying again with the fixed code...")
            attempt += 1   

In [61]:
code="""
print("Hello world)
"""

In [62]:
run_code(code)

(False,
 '  File "<string>", line 2\r\n    print("Hello world)\r\n                       ^\r\nSyntaxError: EOL while scanning string literal\r\n')

In [63]:
debug_code(code)

Attempt 1: Running code...
Suggested fix applied:


Generated Code Fix:
  Sure! Here's a fixed version of the code:
```
print("Hello, world!")
```
The change I made was adding a comma after "Hello" to separate the two strings. This will fix the syntax error and allow the code to run correctly.

Trying again with the fixed code...
Attempt 2: Running code...
Suggested fix applied:


Generated Code Fix:
  Sure! Here's a fixed version of the code:
```
print("Hello, world!")
```
The change I made was adding a comma after "Hello" to separate the two strings. This will fix the syntax error and allow the code to run correctly.

I hope this helps! Let me know if you have any questions or if you'd like me to help with anything else.

Trying again with the fixed code...
